In [3]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.util import ngrams
import re
%matplotlib inline

In [22]:
# Importing the text the lazy way.
gatsby="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

gatsby = re.sub(r'--', ' ', gatsby)

print(gatsby)

# We want to use the standard english-language parser.
parser = spacy.load('en')

# Parsing Gatsby.
gatsby = parser(gatsby)

sentences=[sent.text for sent in list(gatsby.sents)]

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=False, 
                          stop_words='english',
                          ngram_range=(1, 1), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. "Whenever you feel like criticizing any one," he told me, "just remember that all the people in this world haven't had the advantages that you've had." He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intima

In [26]:
data_counts

<18x196 sparse matrix of type '<class 'numpy.float64'>'
	with 210 stored elements in Compressed Sparse Row format>

In [23]:
# Calculating similarity
similarity = data_counts * data_counts.T

# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks=nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(ranks.items(), key=lambda x: x[1], reverse=True)
ranked = [(sentences[x[0]], x[1]) for x in ranked]
ranked

[("In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since.",
  0.05971454770349938),
 ('No Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men.',
  0.057124211288780036),
 ("but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that.",
  0.05703463958380199),
 ('The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men.',
  0.056429913139782725),
 ('Reserving judgments is a matter of infinite hope.', 0.05639481220707985),
 ("He didn't say any more", 0.05555555555555555),
 ('at least the terms in which they e

# Keyword summarization

In [24]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words=set(gatsby_filt)

#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
i = 0
for word in gatsby:
    # Checking if any of the word's next four neighbors are in the word list 
    if word in gatsby_filt:
        # The potential neighbors.
        nextwords=gatsby[i+1:i+5]
        # Filtering the neighbors to select only those in the word list
        neighbors=[x for x in nextwords if x in gatsby_filt]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1
    i+=1

print('done!')

done!


In [25]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.values)
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked[:5])

[(0.013816453726267205, hope), (0.012630413374483035, promises), (0.012630413374483035, exempt), (0.012546629344487583, glimpses), (0.012263221036936283, sort)]


# Using bigrams

In [19]:
# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=False, 
                          stop_words='english',
                          ngram_range=(2, 2), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

In [20]:
data_counts

<18x198 sparse matrix of type '<class 'numpy.float64'>'
	with 198 stored elements in Compressed Sparse Row format>

In [21]:
# Calculating similarity
similarity = data_counts * data_counts.T

# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks=nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(ranks.items(), key=lambda x: x[1], reverse=True)
ranked = [(sentences[x[0]], x[1]) for x in ranked]
ranked

[("In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since.",
  0.05555555555555555),
 ('"Whenever you feel like criticizing any one," he told me, "just remember that all the people in this world haven\'t had the advantages that you\'ve had."',
  0.05555555555555555),
 ("He didn't say any more", 0.05555555555555555),
 ("but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that.",
  0.05555555555555555),
 ("In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores.",
  0.05555555555555555),
 ('The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men.',
  0.05555555

In [10]:
token_to_text = lambda x: x.text

gatsby_bigram = ''
for sent in sentences:
    token = parser(sent)
    bigram = list(ngrams(token, 2))
    bigram = [list(x) for x in bigram]
    bigram = [list(map(token_to_text, x)) for x in bigram]
    bigram = [' '.join(x) for x in bigram]
    bigram = ' '.join(bigram)
    gatsby_bigram += bigram

gatsby_bigram = parser(gatsby_bigram)

In [11]:
# Removing stop words and punctuation, then getting a list of all unique words in the text
gatsby_bigram_filt = [word for word in gatsby_bigram if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
words=set(gatsby_bigram_filt)

In [12]:
#Creating a grid indicating whether words are within 4 places of the target word
adjacency=pd.DataFrame(columns=words,index=words,data=0)

#Iterating through each word in the text and indicating which of the unique words are its neighbors
i = 0
for word in gatsby_bigram:
    # Checking if any of the word's next four neighbors are in the word list 
    if word in gatsby_bigram_filt:
        # The potential neighbors.
        nextwords=gatsby_bigram[i+1:i+6]
        # Filtering the neighbors to select only those in the word list
        neighbors=[x for x in nextwords if x in gatsby_bigram_filt]
        # Adding 1 to the adjacency matrix for neighbors of the target word
        if neighbors:
            adjacency.loc[word,neighbors]=adjacency.loc[word,neighbors]+1
    i+=1

print('done!')

done!


In [13]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.values)
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(words)),
                reverse=True)
print(ranked)

[(0.00616288140173892, young), (0.005986988865500229, wake), (0.005839904504426379, fundamental), (0.0058267875680989125, machines), (0.0058133957069586495, reserved), (0.005813395706958649, judgments), (0.005564324818240003, excursions), (0.005423017223105649, revelations), (0.005350619390753994, promises), (0.005350619390753994, wet), (0.005337642358719699, young), (0.0053261358524516415, people), (0.00532613585245164, dignified), (0.00532613585245164, impressionability), (0.00532613585245164, world), (0.0052741400349885685, unknown), (0.0052741400349885685, unknown), (0.005162768244537876, successful), (0.005143540471924969, vulnerable), (0.005114389004939023, series), (0.005099444583081447, short), (0.005095985121652911, mind), (0.005034574720615616, moral), (0.005034574720615616, moral), (0.005023804957056184, elations), (0.0050136222786570555, short), (0.005010437824382767, orat), (0.005007488945991267, abnormal), (0.004971564610621592, abnormal), (0.004971564610621592, bores), (